# Automate HCPCS and ICD-10 Check for Prism

Every quarter, the business requests HCPCS and ICD10 Codes to be updated in the master list for **Prism**, **Que**, **Salesforce**, and **Essette**. This script is used to format files to update and create new HCPCS in Salesforce.

This script will format the file given by the business into a format that is acceptable by Salesforce.

## Prerequisites
- No prior knowledge of Python is required to run this script. However, that knowledge will help you understand the logic and syntax of the script. Specifically, we will be using the **pandas** library to transpose the file given by the business to a DataFrame, then we will use that DataFrame to create a new DataFrame with the necessary information. We will also be using the **datetime** library specifically for naming conventions of our file.
- You will need to **rename** the file given by the business to **request.csv**. If this step is not done, then this script **will not run** properly.
- You will need to save this Jupyter Notebook and the file **request.csv** in the same directory, e.g. in the same location on your local machine.
- You will need a method to run Jupyter Notebooks on your local machine. This script assumes Anaconda has already been installed to your local machine. If Anaconda is not installed in your local machine, then you will need to install it via https://www.anaconda.com/docs/getting-started/anaconda/install.

## Overview of the Process
This script will read the files **request.csv** and **salesforce.csv** and load them into a DataFrame using the pandas library. 

Since Salesforce only cares about the following columns, we will also strip the unnecessary columns from **request**. The columns we will be using from **request** are
- **HCPCS**
- **Long Description**
- **Short Description**

We will be using all columns from **salesforce**. After we read the files, we will then join **salesforce** onto **request** via the **HCPCS** column. Finally, we will compare columns to find
1. HCPCS that need to be created in Salesforce
2. HCPCS that need to have Long Description updated in Salesforce
3. HCPCS that need to have Short Description updated in Salesforce
4. HCPCS that need to be marked as Inactive in Salesforce
5. HPCCS that need no changes 

Everything will be outputed to their own separate .csv files.

'''
select *
from hcpc_codes
where modified_ts >= '2025-12-24'
  and modified_ts <  '2025-12-25'
order by modified_ts desc;
 
'''

In [2]:
# Import Libraries
import pandas as pd
from datetime import datetime

today = datetime.today() # Get Today's Date
formatted_date = today.strftime("%m%d%Y") # Strip Today's Date in the MMDDYYYY Format

# Filenames # Replace with your actual file name
master_hcpc_file = "Master List 12-11-2025 Version 2.csv"
master_icd10_file = "icd10cm_codes_2026.txt"
prism_hcpcs_file = "updatedHcpcsUat.csv"
prism_trans_inds_file = "hcpc_trans_inds_202510151722_CR.csv"
prism_icd10_file = "Prism-ICD10-Staging.csv"

In [3]:
# Load the CSV files into a DataFrame
master_hcpc = pd.read_csv(master_hcpc_file, encoding="cp1252", dtype=str)
master_icd10 = pd.read_csv(master_icd10_file,
                           sep = "\t",
                           header = None,
                           names = ["raw"],
                           dtype = str)
prism_hcpcs = pd.read_csv(prism_hcpcs_file, encoding="cp1252", dtype=str) # Replace with your actual file name
prism_trans_inds = pd.read_csv(prism_trans_inds_file, encoding = "cp1252", dtype = str) # Replace with your actual file name
prism_icd10 = pd.read_csv(prism_icd10_file, dtype = str)

In [4]:
# Clean the Data
# HCPCS Master List Clean Up
master_hcpc["HCPCS"] = master_hcpc["HCPCS"].str.replace(r"[^a-zA-Z0-9 ,]", "", regex=True)
master_hcpc["Long Description"] = master_hcpc["Long Description"].str.replace(r"[^a-zA-Z0-9 ,]", "", regex=True)
master_hcpc["Short Description"] = master_hcpc["Short Description"].str.replace(r"[^a-zA-Z0-9 ,]", "", regex=True)

# ICD10 Master List Clean Up
# Split the whitespace run between Code and Description
master_icd10[["Code", "Description"]] = master_icd10["raw"].str.strip().str.split(r"\s+",
                                                                                 n = 1,
                                                                                 expand = True)
# Drop the raw Column
master_icd10 = master_icd10.drop(columns = ["raw"])

In [5]:
# Using Set Logic, we'll compare the sets of Master List HCPCS vs Prism HCPCS
codes_in_master = set(master_hcpc["HCPCS"])
codes_in_prism = set(prism_hcpcs["hcpc_code"])

only_in_master = codes_in_master - codes_in_prism
only_in_prism = codes_in_prism - codes_in_master
in_both = codes_in_master & codes_in_prism

In [6]:
only_in_master_df = master_hcpc[master_hcpc["HCPCS"].isin(only_in_master)]


In [7]:
print("There are " + str(len(only_in_master)) + " HCPCS that need to be uploaded to Prism")
print("")
print("There are " + str(len(only_in_prism)) + " HCPCS that are in Prism that are Not in the Master List")

There are 0 HCPCS that need to be uploaded to Prism

There are 0 HCPCS that are in Prism that are Not in the Master List


In [8]:
# Output the DataFrames to CSV Files

only_in_master_df.to_csv("Fix_HCPCS_In_Prism-v2.csv", index = False)
#new_hcpcs.to_csv("New_HCPCS.csv", index = False)

# Check Prism Transition Categories

In [10]:
# Merge HCPCS and Transitions to get the HCPCS Code
merged_prism_hcpcs_transition = prism_hcpcs.merge(prism_trans_inds, 
                                            on = "hcpc_code_id", 
                                            how = "left", 
                                            suffixes = ('_hcpcs_table', '_transition_table'))

# Create Transition Categories
# Master List Copy
master_transition = pd.DataFrame({
    "HCPCS": master_hcpc["HCPCS"],
    "Mod1": None,
    "HCPCS Description": master_hcpc["Long Description"],
    "HCPCS Category": master_hcpc["HCPCS Product Category Parent"],
    "Transition Codes": master_hcpc["Transition Categories"],
    "Monthly Frequency": master_hcpc["Frequency"]
})

master_transition["Monthly Frequency"] = master_transition["Monthly Frequency"].apply(
    lambda x: "Y" if str(x).strip().upper() == "MONTHLY" else "N"
)

In [11]:
# Using Set Logic, we'll compare the sets of Master List vs Prism
codes_in_master = set(master_transition["HCPCS"])
codes_in_prism = set(merged_prism_hcpcs_transition["hcpc_code"])

only_in_master = codes_in_master - codes_in_prism
only_in_prism = codes_in_prism - codes_in_master
in_both = codes_in_master & codes_in_prism

In [12]:
# Print Results
print("There are " + str(len(only_in_master)) + " Transition Categories that need to be uploaded to Prism")
print("")
print("There are " + str(len(only_in_prism)) + " Transition Categories that are in Prism that are Not in the Master List")

There are 0 Transition Categories that need to be uploaded to Prism

There are 0 Transition Categories that are in Prism that are Not in the Master List


In [13]:
# Create a DataFrame of Transition Categories in Master List that are NOT in Prism
only_in_master_df = master_transition[master_transition["HCPCS"].isin(only_in_master)]

In [14]:
# Output the Transition Categories that Need Fixes to CSV Files

only_in_master_df.to_csv("Fix_Transition_Categories_In_Prism.csv", index = False)
#new_hcpcs.to_csv("New_HCPCS.csv", index = False)

# ICD-10 Check

We will now undergo a check on the Master List ICD-10 Codes vs which ICD-10 Codes have been uploaded to Prism. We will be using another **Set** comparison between Master List AND Prism.

**Additional Notes**: 
- The table name in Prism MDM that houses the ICD-10 Codes is `list_dtls`
- The column name `listl_dtl_code` maps to **Code** in the Master List
- The column name `list_dtl_desc` maps to **Description** in the Master List
- The column name `dtl_id` maps to the **URL** in the Prism UI

In [16]:
# Using Set Logic, we'll compare the sets of Master List vs Prism
codes_in_master = set(master_icd10["Code"])
codes_in_prism = set(prism_icd10["list_dtl_code"])

only_in_master = codes_in_master - codes_in_prism
only_in_prism = codes_in_prism - codes_in_master
in_both = codes_in_master & codes_in_prism

In [17]:
# Print Results
print("There are " + str(len(only_in_master)) + " ICD-10 Codes that need to be uploaded to Prism")
print("")
print("There are " + str(len(only_in_prism)) + " Transition Categories that are in Prism that are Not in the Master List")

There are 0 ICD-10 Codes that need to be uploaded to Prism

There are 2890 Transition Categories that are in Prism that are Not in the Master List
